# Lab 2: Homography estimation and applications

In this lab you will learn how to estimate a homography relating two images given a set of correspondences between them. Then, you will work with different computer vision applications of the homography.

More precisely, the goals are:

1) Homography estimation with the DLT algorithm. <br>

2) Application: Image mosaics. <br>

3) Refinement of the estimated homography with the Gold Standard algorithm. <br>

4) Application: Camera calibration with a planar pattern and augmented reality. <br>

5) Application: Logo detection. <br>

6) Application: Logo replacement.

The following file combines some text cells (Markdown cells) and code cells. Some parts of the code need to be completed. All tasks you need to complete are marked in <span style='color:Green'> green.  </span>

In [1]:
%%capture
cd "LAB 2"

In [2]:
import numpy as np
import cv2
import math
import sys
import matplotlib
from matplotlib import pyplot as plt
from operator import itemgetter
from utils import apply_H_fixed_image_size, plot_img, Ransac_DLT_homography

## **1. Homography estimation with the DLT algorithm**

### **1.1 Compute image correspondences**
Execute the following code to find image correspondences using ORB [1] (you may also use SIFT, SURF, etc)

[1] Ethan Rublee, Vincent Rabaud, Kurt Konolige, Gary R. Bradski: ORB: An efficient alternative to SIFT or SURF. ICCV 2011: 2564-2571.

In [ ]:
import cv2
import matplotlib.pyplot as plt

def get_matches(img1, img2, img3, ratio_threshold=0.85):
    """
    Computes the good matches between three images (img1, img2, img3) using ORB and the ratio test.
    
    Args:
        img1: The first image.
        img2: The second image.
        img3: The third image.
        ratio_threshold: The threshold to apply for the ratio test.
    
    Returns:
        good_matches_12: The list of good matches between img1 and img2.
        good_matches_23: The list of good matches between img2 and img3.
        kp1, kp2, kp3: Keypoints from the first, second, and third images.
    """
    # Initiate ORB detector
    orb = cv2.ORB_create()
    
    # Find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    kp3, des3 = orb.detectAndCompute(img3, None)
    
    # Keypoint matching
    bf = cv2.BFMatcher()

    # Matching between img1 and img2
    matches_12 = bf.knnMatch(des1, des2, k=2)
    good_matches_12 = []
    for m, n in matches_12:
        if m.distance < ratio_threshold * n.distance:
            good_matches_12.append([m])

    # Matching between img2 and img3
    matches_23 = bf.knnMatch(des2, des3, k=2)
    good_matches_23 = []
    for m, n in matches_23:
        if m.distance < ratio_threshold * n.distance:
            good_matches_23.append([m])

    return good_matches_12, good_matches_23, kp1, kp2, kp3

# Example usage for 3 images
img1 = cv2.imread('Data/llanes/llanes_a.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/llanes/llanes_b.jpg', cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread('Data/llanes/llanes_c.jpg', cv2.IMREAD_GRAYSCALE)

good_matches_12, good_matches_23, kp1, kp2, kp3 = get_matches(img1, img2, img3)

# Show "good" matches between img1 and img2
img_12 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches_12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_12)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

# Show "good" matches between img2 and img3
img_23 = cv2.drawMatchesKnn(img2, kp2, img3, kp3, good_matches_23, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_23)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

### **1.2 Compute the homography (DLT algorithm) between image pairs**

The following cell of code calls the 'Ransac_DLT_homography' function from utils.py
Before running the code of the cell you first need to complete some functions in utils.py that are called from 'Ransac_DLT_homography' function:

<span style='color:Green'> - Complete the 'DLT_homography' function that computes a homography from a set of point correspondences with the DLT algorithm.  </span>

<span style='color:Green'> - Complete the 'Inliers' function that, given a homography, a set of point correspondences and a certain threshold  returns the indices of the correspondences that are inliers.  </span>


In [ ]:
def get_homography(kp1, kp2, good_matches, th = 3, max_it = 1000):
    """
    Computes the homography between two images given keypoints and matches.
    
    Args:
        kp1: Keypoints from the first image.
        kp2: Keypoints from the second image.
        good_matches: List of good matches between the images.
    
    Returns:
        H: The computed homography matrix.
        inlier_matches: List of inlier matches.
    """
    points1 = []
    points2 = []
    
    # Extract points from the good matches
    for m in good_matches:
        points1.append([kp1[m[0].queryIdx].pt[0], kp1[m[0].queryIdx].pt[1], 1])
        points2.append([kp2[m[0].trainIdx].pt[0], kp2[m[0].trainIdx].pt[1], 1])
    
    points1 = np.asarray(points1).T
    points2 = np.asarray(points2).T
    
    # Compute homography using RANSAC
    H, indices_inlier_matches = Ransac_DLT_homography(points1, points2, th, max_it)
    
    # Get inlier matches
    inlier_matches = itemgetter(*indices_inlier_matches)(good_matches)
    
    return H, inlier_matches

# Example usage for homography between images 1 and 2
H_12, inlier_matches_12 = get_homography(kp1, kp2, good_matches_12)

# Visualize matches for image 1 and 2
img_12 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, inlier_matches_12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_12)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

# Example usage for homography between images 2 and 3
H_23, inlier_matches_23 = get_homography(kp2, kp3, good_matches_23)

# Visualize matches for image 2 and 3
img_23 = cv2.drawMatchesKnn(img2, kp2, img3, kp3, inlier_matches_23, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_23)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

## **2. Application: Image mosaics**

Now that the homographies relating the images have been estimated we can create a mosaic by properly transforming and fusing the different image contents.
For that we create a big enough canvas where the mosaic will be created. The function 'apply_H_fixed_image_size' is provided in utils.py, it is a variant of the 'apply_H' function from lab 1 where this time the size of the transformed is fixed and given as an extra input.

<span style='color:Green'> - Complete the 2nd input argument to the 'apply_H_fixed_image_size' function in order to get the image mosaic. </span>

In [5]:
img1 = cv2.imread('Data/llanes/llanes_a.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/llanes/llanes_b.jpg', cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread('Data/llanes/llanes_c.jpg', cv2.IMREAD_GRAYSCALE)

In [ ]:
good_matches_12, good_matches_23, kp1, kp2, kp3 = get_matches(img1, img2, img3)

img_12 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches_12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_12)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_23 = cv2.drawMatchesKnn(img2, kp2, img3, kp3, good_matches_23, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_23)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
H_12, inlier_matches_12 = get_homography(kp1, kp2, good_matches_12, th=1)
H_23, inlier_matches_23 = get_homography(kp2, kp3, good_matches_23, th=1)
print(len(inlier_matches_12), len(inlier_matches_23))

img_12 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, inlier_matches_12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_12)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_23 = cv2.drawMatchesKnn(img2, kp2, img3, kp3, inlier_matches_23, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_23)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
corners = [-400, 1200, -100, 650] # corners of the canvas where the mosaic will be created

img1c = cv2.imread('Data/llanes/llanes_a.jpg', cv2.IMREAD_COLOR)
img2c = cv2.imread('Data/llanes/llanes_b.jpg', cv2.IMREAD_COLOR)
img3c = cv2.imread('Data/llanes/llanes_c.jpg', cv2.IMREAD_COLOR)
img1c = cv2.cvtColor(img1c, cv2.COLOR_BGR2RGB)
img2c = cv2.cvtColor(img2c, cv2.COLOR_BGR2RGB)
img3c = cv2.cvtColor(img3c, cv2.COLOR_BGR2RGB)

img1c_w = apply_H_fixed_image_size(img1c, H_12, corners) # complete the call to the function
img2c_w = apply_H_fixed_image_size(img2c, np.eye(3), corners) # complete the call to the function
img3c_w = apply_H_fixed_image_size(img3c, np.linalg.inv(H_23), corners) # complete the call to the function

img_mosaic = np.maximum(img3c_w,np.maximum(img1c_w,img2c_w))
plot_img(img_mosaic)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
img_mosaic = cv2.cvtColor(img_mosaic, cv2.COLOR_RGB2BGR)
cv2.imwrite('mosaic_llanes.png', img_mosaic) 

<span style='color:Green'> - Compute the mosaic with images from folder 'castle_int'. </span>

In [9]:
img1 = cv2.imread('Data/castle_int/0014_s.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/castle_int/0015_s.png', cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread('Data/castle_int/0016_s.png', cv2.IMREAD_GRAYSCALE)

In [ ]:
good_matches_12, good_matches_23, kp1, kp2, kp3 = get_matches(img1, img2, img3)

img_12 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches_12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_12)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_23 = cv2.drawMatchesKnn(img2, kp2, img3, kp3, good_matches_23, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_23)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
H_12, inlier_matches_12 = get_homography(kp1, kp2, good_matches_12, th=10)
H_23, inlier_matches_23 = get_homography(kp2, kp3, good_matches_23, th=10)
print(len(inlier_matches_12), len(inlier_matches_23))

img_12 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, inlier_matches_12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_12)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_23 = cv2.drawMatchesKnn(img2, kp2, img3, kp3, inlier_matches_23, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_23)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
corners = [-400, 1200, -100, 650] # corners of the canvas where the mosaic will be created

img1c = cv2.imread('Data/castle_int/0014_s.png', cv2.IMREAD_COLOR)
img2c = cv2.imread('Data/castle_int/0015_s.png', cv2.IMREAD_COLOR)
img3c = cv2.imread('Data/castle_int/0016_s.png', cv2.IMREAD_COLOR)
img1c = cv2.cvtColor(img1c, cv2.COLOR_BGR2RGB)
img2c = cv2.cvtColor(img2c, cv2.COLOR_BGR2RGB)
img3c = cv2.cvtColor(img3c, cv2.COLOR_BGR2RGB)

img1c_w = apply_H_fixed_image_size(img1c, H_12, corners) # complete the call to the function
img2c_w = apply_H_fixed_image_size(img2c, np.eye(3), corners) # complete the call to the function
img3c_w = apply_H_fixed_image_size(img3c, np.linalg.inv(H_23), corners) # complete the call to the function

img_mosaic = np.maximum(img3c_w,np.maximum(img1c_w,img2c_w))
plot_img(img_mosaic)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
img_mosaic = cv2.cvtColor(img_mosaic, cv2.COLOR_RGB2BGR)
cv2.imwrite('mosaic_castle_int.png', img_mosaic)

<span style='color:Green'> - Compute the mosaic with images from folder 'aerial'. </span>

In [13]:
img1 = cv2.imread('Data/aerial/frame_00001.tif', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/aerial/frame_00018.tif', cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread('Data/aerial/frame_00030.tif', cv2.IMREAD_GRAYSCALE)

In [ ]:
good_matches_12, good_matches_23, kp1, kp2, kp3 = get_matches(img1, img2, img3, ratio_threshold=0.95)

img_12 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches_12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_12)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_23 = cv2.drawMatchesKnn(img2, kp2, img3, kp3, good_matches_23, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_23)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
H_12, inlier_matches_12 = get_homography(kp1, kp2, good_matches_12, th=10)
H_23, inlier_matches_23 = get_homography(kp2, kp3, good_matches_23, th=10)
print(len(inlier_matches_12), len(inlier_matches_23))

img_12 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, inlier_matches_12, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_12)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_23 = cv2.drawMatchesKnn(img2, kp2, img3, kp3, inlier_matches_23, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_23)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
corners = [-400, 1200, -100, 650] # corners of the canvas where the mosaic will be created

img1c = cv2.imread('Data/aerial/frame_00001.tif', cv2.IMREAD_COLOR)
img2c = cv2.imread('Data/aerial/frame_00018.tif', cv2.IMREAD_COLOR)
img3c = cv2.imread('Data/aerial/frame_00030.tif', cv2.IMREAD_COLOR)
img1c = cv2.cvtColor(img1c, cv2.COLOR_BGR2RGB)
img2c = cv2.cvtColor(img2c, cv2.COLOR_BGR2RGB)
img3c = cv2.cvtColor(img3c, cv2.COLOR_BGR2RGB)

img1c_w = apply_H_fixed_image_size(img1c, H_12, corners) # complete the call to the function
img2c_w = apply_H_fixed_image_size(img2c, np.eye(3), corners) # complete the call to the function
img3c_w = apply_H_fixed_image_size(img3c, np.linalg.inv(H_23), corners) # complete the call to the function

img_mosaic = np.maximum(img3c_w,np.maximum(img1c_w,img2c_w))
plot_img(img_mosaic)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
img_mosaic = cv2.cvtColor(img_mosaic, cv2.COLOR_RGB2BGR)
cv2.imwrite('mosaic_castle_int.png', img_mosaic)

<span style='color:Green'> - Compute the mosaic with 5 images from folder 'ny'. </span>

In [23]:
img1 = cv2.imread('Data/ny/NYs1.jpeg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('Data/ny/NYs2.jpeg', cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread('Data/ny/NYs3.jpeg', cv2.IMREAD_GRAYSCALE)  # we will project onto image 3
img4 = cv2.imread('Data/ny/NYs4.jpeg', cv2.IMREAD_GRAYSCALE)
img5 = cv2.imread('Data/ny/NYs5.jpeg', cv2.IMREAD_GRAYSCALE)

In [ ]:
good_matches_13, good_matches_32, kp1, kp3, kp2 = get_matches(img1, img3, img2, ratio_threshold=0.95)
good_matches_43, good_matches_35, kp4, kp3, kp5 = get_matches(img4, img3, img5, ratio_threshold=0.95)

img_13 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches_13, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_13)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_32 = cv2.drawMatchesKnn(img3, kp3, img2, kp2, good_matches_32, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_32)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_43 = cv2.drawMatchesKnn(img4, kp4, img3, kp3, good_matches_43, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_43)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_35 = cv2.drawMatchesKnn(img3, kp3, img5, kp5, good_matches_35, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_35)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
H_13, inlier_matches_13 = get_homography(kp1, kp3, good_matches_13, th=10)
H_32, inlier_matches_32 = get_homography(kp3, kp2, good_matches_32, th=10)
H_43, inlier_matches_43 = get_homography(kp4, kp3, good_matches_43, th=10)
H_35, inlier_matches_35 = get_homography(kp3, kp5, good_matches_35, th=10)

img_13 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, inlier_matches_13, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_13)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_32 = cv2.drawMatchesKnn(img3, kp3, img2, kp2, inlier_matches_32, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_32)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_43 = cv2.drawMatchesKnn(img4, kp4, img3, kp3, inlier_matches_43, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_43)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

img_35 = cv2.drawMatchesKnn(img3, kp3, img5, kp5, inlier_matches_35, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img_35)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
corners = [-1000, 2000, -400, 800] # corners of the canvas where the mosaic will be created

img1c = cv2.imread('Data/ny/NYs1.jpeg', cv2.IMREAD_COLOR)
img2c = cv2.imread('Data/ny/NYs2.jpeg', cv2.IMREAD_COLOR)
img3c = cv2.imread('Data/ny/NYs3.jpeg', cv2.IMREAD_COLOR)  # we will project onto image 3
img4c = cv2.imread('Data/ny/NYs4.jpeg', cv2.IMREAD_COLOR)
img5c = cv2.imread('Data/ny/NYs5.jpeg', cv2.IMREAD_COLOR)
img1c = cv2.cvtColor(img1c, cv2.COLOR_BGR2RGB)
img2c = cv2.cvtColor(img2c, cv2.COLOR_BGR2RGB)
img3c = cv2.cvtColor(img3c, cv2.COLOR_BGR2RGB)
img4c = cv2.cvtColor(img4c, cv2.COLOR_BGR2RGB)
img5c = cv2.cvtColor(img5c, cv2.COLOR_BGR2RGB)

img1c_w = apply_H_fixed_image_size(img1c, H_13, corners) # complete the call to the function
img2c_w = apply_H_fixed_image_size(img2c, np.linalg.inv(H_32), corners) # complete the call to the function
img3c_w = apply_H_fixed_image_size(img3c, np.eye(3), corners) # complete the call to the function
img4c_w = apply_H_fixed_image_size(img4c, H_43, corners) # complete the call to the function
img5c_w = apply_H_fixed_image_size(img5c, np.linalg.inv(H_35), corners) # complete the call to the function

img_mosaic = np.maximum(img5c_w,np.maximum(img4c_w,np.maximum(img3c_w,np.maximum(img1c_w,img2c_w))))
plot_img(img_mosaic)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
img_mosaic = cv2.cvtColor(img_mosaic, cv2.COLOR_RGB2BGR)
cv2.imwrite('mosaic_castle_int.png', img_mosaic)

<span style='color:Green'> - In the report, comment the results in every of the four cases: hypothetise why it works or does not work </span>

## **3. Refinement of the estimated homography with the Gold Standard algorithm**

In order to refine the previous estimated homography we can minimize the geometric error with the Levenberg-Marquardt algorithm. For that we will call the function 'least_squares' with the proper input arguments that you have to complete. First, it is recommended to read the documentation of that function:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html

Notice that the first input is a function that only provides the vector of the residuals. The algorithm constructs the cost function as a sum of squares of the residuals. In our case the function is called 'geometric_error_terms' and you need to complete it.

More precisely, the tasks to perform are:

<span style='color:Green'> - Complete the function 'geometric_error_terms'. </span>

In [9]:
from scipy.optimize import least_squares

def geometric_error_terms(variables, data_points):  
    # Extract the homography from the flattened variables
    H = variables.reshape(3, 3)
    
    # Extract points from data_points
    points1 = data_points[0]  # Points from the first image (N, 2)
    points2 = data_points[1]  # Points from the second image (N, 2)
    
    # Convert points1 to homogeneous coordinates (N, 3)
    points1_homog = np.hstack((points1, np.ones((points1.shape[0], 1))))  # (N, 3)
    
    # Transform points1 using the homography H to get the corresponding points in image 2
    points1_transformed = np.dot(points1_homog, H.T)  # (N, 3)
    
    # Normalize the transformed points (convert back from homogeneous coordinates)
    points1_transformed /= points1_transformed[:, 2:3]  # Normalize by the z-component
    
    # Calculate residuals (difference between transformed points and target points)
    residuals = points1_transformed[:, :2] - points2
    
    # Flatten the residuals into a single array to return
    return residuals.flatten()

<span style='color:Green'> - Complete the code before calling function 'geometric_error_terms' in order to create the proper input of variables, 'variables0', to pass to the function. </span>

In [10]:
H_12_initial = H_12.copy()
variables0 = H_12_initial.flatten()
points1 = np.array([kp.pt for kp in kp1])
points2 = np.array([kp.pt for kp in kp2])

<span style='color:Green'> - Extract the refined homogaphy and the refined keypoint locations from the output, 'result', of function 'geometric_error_terms'.  </span>

In [11]:
result = least_squares(geometric_error_terms, variables0, method='lm', args=([points1, points2], ))
H_12_refined = result.x.reshape(3, 3)

<span style='color:Green'> - To better compare the two homographies (refined and non refined) compute and print the geometric error before and after the refinement. </span>

In [ ]:
# Calculate and print the geometric error before and after the refinement
# Initial error
points1_homog = np.hstack((points1, np.ones((points1.shape[0], 1))))  # Convert to homogeneous coordinates (500, 3)
points1_transformed_initial = np.dot(points1_homog, H_12_initial.T)  # (500, 3)
points1_transformed_initial /= points1_transformed_initial[:, 2:3]  # Normalize by the z-component
initial_error = np.sum(np.linalg.norm(points1_transformed_initial[:, :2] - points2, axis=1))

# Refined error
points1_transformed_refined = np.dot(points1_homog, H_12_refined.T)  # (500, 3)
points1_transformed_refined /= points1_transformed_refined[:, 2:3]  # Normalize by the z-component
refined_error = np.sum(np.linalg.norm(points1_transformed_refined[:, :2] - points2, axis=1))

# Print errors
print(f"Initial geometric error: {initial_error}")
print(f"Refined geometric error: {refined_error}")

# Extract refined keypoints by applying the refined homography to points1
points1_refined_homog = np.hstack((points1, np.ones((points1.shape[0], 1))))  # Convert to homogeneous coordinates
points1_refined_transformed = np.dot(points1_refined_homog, H_12_refined.T)  # (500, 3)
points1_refined_transformed /= points1_refined_transformed[:, 2:3]  # Normalize by the z-component
points1_refined = points1_refined_transformed[:, :2]

# Display points and refined points on images
img1c = plt.imread('Data/llanes/llanes_a.jpg')  # Example, replace with actual image
img2c = plt.imread('Data/llanes/llanes_b.jpg')  # Example, replace with actual image

fig, ax = plt.subplots()  # image 1
ax.imshow(img1c)
ax.scatter(points1[:, 0], points1[:, 1], c='cyan', marker='+')
ax.scatter(points1_refined[:, 0], points1_refined[:, 1], c='fuchsia', marker='+')
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

fig, ax = plt.subplots()  # image 2
ax.imshow(img2c)
ax.scatter(points2[:, 0], points2[:, 1], c='cyan', marker='+')
ax.scatter(points1_refined[:, 0], points1_refined[:, 1], c='fuchsia', marker='+')
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

<span style='color:Green'> - Refine the homography 23 with the Gold Standard algorithm and visualize the differences in the keypoint locations. </span>

In [13]:
H_23_initial = H_23.copy()
variables0 = H_23_initial.flatten()
points2 = np.array([kp.pt for kp in kp2])
points3 = np.array([kp.pt for kp in kp3])
result = least_squares(geometric_error_terms, variables0, method='lm', args=([points2, points3], ))
H_23_refined = result.x.reshape(3, 3)

In [ ]:
# Calculate and print the geometric error before and after the refinement
# Initial error
points2_homog = np.hstack((points2, np.ones((points2.shape[0], 1))))  # Convert to homogeneous coordinates (500, 3)
points2_transformed_initial = np.dot(points2_homog, H_23_initial.T)  # (500, 3)
points2_transformed_initial /= points2_transformed_initial[:, 2:3]  # Normalize by the z-component
initial_error_23 = np.sum(np.linalg.norm(points2_transformed_initial[:, :2] - points3, axis=1))

# Refined error
points2_transformed_refined = np.dot(points2_homog, H_23_refined.T)  # (500, 3)
points2_transformed_refined /= points2_transformed_refined[:, 2:3]  # Normalize by the z-component
refined_error_23 = np.sum(np.linalg.norm(points2_transformed_refined[:, :2] - points3, axis=1))

# Print errors for H_23
print(f"Initial geometric error for H_23: {initial_error_23}")
print(f"Refined geometric error for H_23: {refined_error_23}")

# Extract refined keypoints by applying the refined homography to points2
points2_refined_homog = np.hstack((points2, np.ones((points2.shape[0], 1))))  # Convert to homogeneous coordinates
points2_refined_transformed = np.dot(points2_refined_homog, H_23_refined.T)  # (500, 3)
points2_refined_transformed /= points2_refined_transformed[:, 2:3]  # Normalize by the z-component
points2_refined = points2_refined_transformed[:, :2]

# Display points and refined points on images
img2c = plt.imread('Data/llanes/llanes_b.jpg')  # Example, replace with actual image
img3c = plt.imread('Data/llanes/llanes_c.jpg')  # Example, replace with actual image

# Plot results for image 2
fig, ax = plt.subplots()  # image 2
ax.imshow(img2c)
ax.scatter(points2[:, 0], points2[:, 1], c='cyan', marker='+')
ax.scatter(points2_refined[:, 0], points2_refined[:, 1], c='fuchsia', marker='+')
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

# Plot results for image 3
fig, ax = plt.subplots()  # image 3
ax.imshow(img3c)
ax.scatter(points3[:, 0], points3[:, 1], c='cyan', marker='+')
ax.scatter(points2_refined[:, 0], points2_refined[:, 1], c='fuchsia', marker='+')
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

<span style='color:Green'> - Build mosaic with the refined homographies. </span>

In [ ]:
corners = [-400, 1200, -100, 650] # corners of the canvas where the mosaic will be created

img1c = cv2.imread('Data/llanes/llanes_a.jpg', cv2.IMREAD_COLOR)
img2c = cv2.imread('Data/llanes/llanes_b.jpg', cv2.IMREAD_COLOR)
img3c = cv2.imread('Data/llanes/llanes_c.jpg', cv2.IMREAD_COLOR)
img1c = cv2.cvtColor(img1c, cv2.COLOR_BGR2RGB)
img2c = cv2.cvtColor(img2c, cv2.COLOR_BGR2RGB)
img3c = cv2.cvtColor(img3c, cv2.COLOR_BGR2RGB)

img1c_w = apply_H_fixed_image_size(img1c, H_12, corners) # complete the call to the function
img2c_w = apply_H_fixed_image_size(img2c, np.eye(3), corners) # complete the call to the function
img3c_w = apply_H_fixed_image_size(img3c, np.linalg.inv(H_23), corners) # complete the call to the function

img_mosaic = np.maximum(img3c_w,np.maximum(img1c_w,img2c_w))
plot_img(img_mosaic)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
img_mosaic = cv2.cvtColor(img_mosaic, cv2.COLOR_RGB2BGR)
cv2.imwrite('mosaic_llanes.png', img_mosaic) 

In [ ]:
corners = [-400, 1200, -100, 650] # corners of the canvas where the mosaic will be created

img1c = cv2.imread('Data/llanes/llanes_a.jpg', cv2.IMREAD_COLOR)
img2c = cv2.imread('Data/llanes/llanes_b.jpg', cv2.IMREAD_COLOR)
img3c = cv2.imread('Data/llanes/llanes_c.jpg', cv2.IMREAD_COLOR)
img1c = cv2.cvtColor(img1c, cv2.COLOR_BGR2RGB)
img2c = cv2.cvtColor(img2c, cv2.COLOR_BGR2RGB)
img3c = cv2.cvtColor(img3c, cv2.COLOR_BGR2RGB)

img1c_w = apply_H_fixed_image_size(img1c, H_12_refined, corners) # complete the call to the function
img2c_w = apply_H_fixed_image_size(img2c, np.eye(3), corners) # complete the call to the function
img3c_w = apply_H_fixed_image_size(img3c, np.linalg.inv(H_23_refined), corners) # complete the call to the function

img_mosaic = np.maximum(img3c_w,np.maximum(img1c_w,img2c_w))
plot_img(img_mosaic)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
img_mosaic = cv2.cvtColor(img_mosaic, cv2.COLOR_RGB2BGR)
cv2.imwrite('mosaic_llanes.png', img_mosaic) 

## **4. Application: Calibration with a planar pattern and augmented reality**

As we have seen in class, we can calibrate a camera with a planar pattern with Zhang's algorithm. A key aspect of this algorithm is the estimation of a homography that relates a pair of images, in particular, the template image and an image of it taken with the camera we want to calibrate. Once the camera is calibrated and we have recovered the relative pose between the camera and the planar pattern we can properly insert a virtual object on top of the flat pattern in a way that it is consistent, in a perspective sense, with rest of the scene.

### **4.1 Camera calibration**

The following code calibrates a camera using N (where N greater or equal than three) views of the planar pattern. Most of the code is provided but there are parts you need to complete. These parts are indicated in green before the corresponding cell code.


In [ ]:
# Read template image and calibration images
template = cv2.imread('Data/calib/template.jpg',cv2.IMREAD_GRAYSCALE)
images = []
N = 3 # 3, 4, or 5
for i in range(1,N+1):
    m = cv2.imread("Data/calib/graffiti{0}.tif".format(i),cv2.IMREAD_GRAYSCALE)
    images.append(m)

# Initiate ORB detector
orb = cv2.ORB_create()
# find the keypoints and descriptors with ORB
kpt, dest = orb.detectAndCompute(template,None)
kpi, desi = [], []
for m in images:
    kp, des = orb.detectAndCompute(m,None)
    kpi.append(kp)
    desi.append(des)


# Keypoint matching and homography estimation
bf = cv2.BFMatcher()
H = []
for i in range(N):
    matches = bf.knnMatch(dest,desi[i],k=2)
    # Apply ratio test
    good_matches = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good_matches.append([m])
    
    # Fit homography and remove outliers
    points1 = []
    points2 = []
    for m in good_matches:
        points1.append([kpt[m[0].queryIdx].pt[0], kpt[m[0].queryIdx].pt[1], 1])
        points2.append([kpi[i][m[0].trainIdx].pt[0], kpi[i][m[0].trainIdx].pt[1], 1])
    
    points1 = np.asarray(points1)
    points1 = points1.T
    points2 = np.asarray(points2)
    points2 = points2.T
    
    Hi, indices_inlier_matches = Ransac_DLT_homography(points1, points2, 3, 1000)
    H.append(Hi)
    
    # Show inlier matches
    inlier_matches = itemgetter(*indices_inlier_matches)(good_matches)
    img_12 = cv2.drawMatchesKnn(template,kpt,images[i],kpi[i],inlier_matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.imshow(img_12)
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(18.5, 10.5)
    plt.show()
    


<span style='color:Green'> - Complete the code that computes the Image of the Absolute Conic. </span>

<span style='color:Green'> - Complete the code that computes the matrix of internal parameters, K. </span>

Note: there is a linalg.cholesky function both in numpy and scipy, you have to read the documentation of both and find out which is the proper one to use here.

<span style='color:Green'> - In the report, make some comments related to the obtained internal camera parameters and also comment their relation to the image size. </span>

In [ ]:
# Compute the Image of the Absolute Conic ...

# Compute the matrix of internal parameters, K ...


<span style='color:Green'> - Complete the calculation of r1, r2, and ti in the code below. </span>

In [ ]:
# Compute camera position and orientation
import math
R = []
t = []
P = []

for i in range(N):
    # compute r1, r2, and ti
    h = H[i]
    r1 = # complete ...
    r2 = # complete ...
    ti = # complete ...
    
    # Solve the scale ambiguity by forcing r1 and r2 to be unit vectors.
    s = math.sqrt(np.linalg.norm(r1) * np.linalg.norm(r2)) * np.sign(ti[2])
    r1 = r1 / s
    r2 = r2 / s
    ti = ti / s
    t.append(ti)
    Ri = np.array([r1, r2, np.cross(r1,r2)])
    Ri = Ri.T
    
    # Ensure R is a rotation matrix
    U, d, Vt = np.linalg.svd(Ri)
    Ri = U @ np.identity(3) @ Vt
    R.append(Ri)
   
    # Pi = K * [Ri ti]
    A = np.zeros((3,4))
    A[:3,:3] = Ri
    A[:,3] = ti
    Pi = K @ A
    P.append(Pi)

The following code (no need to be completed) draws the planar pattern and the N camera locations (position and orientation).

<span style='color:Green'> - Take a look to the provided code for that and in the report explain how the optical center is computed. </span>

In [ ]:
import plotly.graph_objects as go
from utils import plot_camera, plot_image_origin

ny, nx = images[0].shape

fig = go.Figure()
for i in range(N):
    plot_camera(P[i], nx, ny, fig, "camera{0}".format(i))

plot_image_origin(nx, ny, fig, "image")

fig.show()

Alternatively, we can draw a fixed camera and N planar patterns at different relative poses with respect to the camera.

<span style='color:Green'> -Complete the function ' plot_image_Rt' in order to achieve that. Note: it may be useful to look at function 'plot_image_origin' in utils.py</span>

In [ ]:
def plot_image_Rt(R,t,w, h, fig, legend):
    p1 = # to complete ...
    p2 = # to complete ...
    p3 = # to complete ...
    p4 = # to complete ...

    x = np.array([p1[0], p2[0], p3[0], p4[0], p1[0]])
    y = np.array([p1[1], p2[1], p3[1], p4[1], p1[1]])
    z = np.array([p1[2], p2[2], p3[2], p4[2], p1[2]])

    fig.add_trace(go.Scatter3d(x=x, y=z, z=-y, mode='lines',name=legend))
    
    return


fig = go.Figure()
A = np.zeros((3,4))
A[0,0]=A[1,1]=A[2,2]=1

plot_camera(K@A, nx, ny, fig, "camera")
for i in range(N):
    plot_image_Rt(R[i], t[i], nx, ny, fig, "image{0}".format(i))

fig.show()

### **4.2 OPTIONAL. Augmented reality**

The following code (no need to be completed) draws a simple 3D virtual object (a cube) on top of the flat pattern.

<span style='color:Green'> - Analyse the provided code and in the report explain how the virtual object is inserted in the image and why the camera view needs to be calibrated. </span>

In [ ]:
Th, Tw = template.shape
cube_corners = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0], [0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 1, 1], [0, 1, 1], [0, 0, 1], [0, 0, 0], [1, 0, 0], [1, 0, 1], [0, 0, 1], [0, 0, 0], [0, 1, 0], [1, 1, 0], [1, 1, 1], [0, 1, 1], [0, 1, 0]])

import numpy.matlib
offset = np.array([Tw/2, Th/2, -Tw/8])
M = cube_corners.shape[0]
X = (cube_corners - 0.5) * Tw/ 4 + np.matlib.repmat(offset, M, 1)

X = X.T
ones = np.ones(M)
Xh = np.stack((X[0,:], X[1,:], X[2,:], ones), axis=0)

line_color = (0, 255, 0)

for i in range(N):
    
    xp = P[i]@Xh
    xp = xp / xp[2,:]
    xp = xp.astype(int)
    
    image = cv2.imread("Data/calib/graffiti{0}.tif".format(i+1),cv2.IMREAD_COLOR)
    image_gray = image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray[:, :, 0] = gray[:, :]
    image_gray[:, :, 1] = image_gray[:, :, 0]
    image_gray[:, :, 2] = image_gray[:, :, 0]
    for j in range(M-1):
        image = cv2.line(image_gray, (xp[0,j],xp[1,j]), (xp[0,j+1],xp[1,j+1]), line_color, 4) 
    plt.imshow(image)
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(18.5, 10.5)
    plt.show()


## **5. OPTIONAL. Application: Logo detection**

<span style='color:Green'> - Detect the UPF logo in the two UPF images using the DLT algorithm (folder "logos").
Interpret and comment the results. </span>

## **6. OPTIONAL. Application: Logo replacement**

<span style='color:Green'> - Replace the UPF logo by the master logo in one of the previous images using the DLT algorithm. </span>
